In [ ]:
import requests
import configparser
import pandas as pd
import gspread
from google.colab import drive
from google.colab import auth
from google.colab import userdata
from google.auth import default

In [ ]:
#Authorize
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
#Mount google drive file path
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
#Pull client ID and secret from google storage
client_id = userdata.get('etqdev_clientid')
client_secret = userdata.get('etqdev_secret')

In [ ]:
#define parameters for access token call
url = 'https://reliance-auth-prod-a.auth.us-east-1.amazoncognito.com/oauth2/token'
data = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}
headers = {'Content-Type':'application/x-www-form-urlencoded'}

In [ ]:
#Send post request to get access token
response = requests.post(url,data=data,headers=headers)

In [ ]:
#Check if the request was successful. If successful, process the data
if response.status_code == 200:
    data = response.json()
    access_token = data.get('access_token', None)
    print('Access token aquired')
else:
    print('Failed to retrieve access token:', response.status_code)

Access token aquired


In [ ]:
#define parameters for record get call
application_name = 'CUSTOMERS'
form_name = 'CUSTOMER_COMPLAINTS'
document_IDs = '1637, 1638, 1639, 1640'
base_url = 'https://dev-peloton.api.etq.com/reliance'
full_url = f"{base_url}/rest/v1/documents/{application_name}/{form_name}/{document_IDs}"
headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {access_token}"}

In [ ]:
#Make get call and save resulting json
response = requests.get(full_url, headers=headers)
if response.status_code == 200:
        data = response.json()
        print('Data retrieved successfully')
else:
    print('Failed to retrieve data:', response.status_code)

Data retrieved successfully


In [ ]:
#Normalize json data and save into dataframe to get a list of records
records = pd.json_normalize(data['Document'])

In [ ]:
#Transform the data to match the output of google sheets data source
rows = []
for field in data['Document']:
    row = {}
    for fieldname in field['Fields']:
        name = fieldname['fieldName']
        value = fieldname.get('Values',[None])[0]
        row[name] = value
    rows.append(row)
df = pd.DataFrame(rows)

In [ ]:
#Open database source sheet
sheet = gc.open('Test - Safety Database Source').sheet1

In [ ]:
#Update the sheet with column titles and values from the dataframe
sheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1UDOT0FoI5-2BiQ-TaBvSs3fXC7QEmDPGaQDpVNy6_9E',
 'updatedRange': 'Sheet1!A1:DT5',
 'updatedRows': 5,
 'updatedColumns': 124,
 'updatedCells': 230}

In [ ]:
#Reset index to incident ID column, TBD
#df.set_index('PEL_INCIDENTS_PRIMARY_INC', inplace = True)